#  1. Load Data

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

file = "TRADES_CopyTr_90D_ROI.csv"
data = pd.read_csv(file)

# it display first 5 rows of data
data.head()

print("Initial data shape:", data.shape)
data.head(2)



Initial data shape: (150, 2)


,Port_IDs,Trade_History
0,3925368433214965504,"[{'time': 1718899656000, 'symbol': 'SOLUSDT', ..."
1,4002413037164645377,"[{'time': 1718980078000, 'symbol': 'NEARUSDT',..."


# 2. Process Trade History

In [5]:
# Convert trade history to Python objects
parsed_trades = []

for idx, row in data.iterrows():
    port_id = row['Port_IDs']
    trade_history = row['Trade_History']
    
    # Skip NaN values
    if isinstance(trade_history, float):
        print(f"Skipping invalid trade history for Port_ID {port_id}")
        continue
    
    try:
        # Clean and parse the trade history
        trade_str = trade_history.replace("'", '"')
        trades = eval(trade_str)
        
        for trade in trades:
            trade['Port_IDs'] = port_id
            parsed_trades.append(trade)
            
    except Exception as e:
        print(f"Error processing Port_ID {port_id}: {str(e)}")
        continue

trade_df = pd.DataFrame(parsed_trades)
print("\nProcessed trades shape:", trade_df.shape)
trade_df.head(2)

Skipping invalid trade history for Port_ID 3919965573764932864

Processed trades shape: (211277, 15)


,time,symbol,side,price,fee,feeAsset,quantity,quantityAsset,realizedProfit,realizedProfitAsset,baseAsset,qty,positionSide,activeBuy,Port_IDs
0,1718899656000,SOLUSDT,BUY,132.53700,-0.994027,USDT,1988.05500,USDT,0.0,USDT,SOL,15.0,LONG,True,3925368433214965504
1,1718899618000,DOGEUSDT,BUY,0.12182,-0.279796,USDT,1398.98088,USDT,0.0,USDT,DOGE,11484.0,LONG,False,3925368433214965504


# 3. Clean Data

In [6]:
numeric_cols = ['price', 'fee', 'quantity', 'realizedProfit', 'qty']
trade_df[numeric_cols] = trade_df[numeric_cols].apply(pd.to_numeric, errors='coerce')
trade_df['time'] = pd.to_datetime(trade_df['time'], unit='ms')

# Classify positions
trade_df['position_type'] = np.select(
    [
        (trade_df['side'] == 'BUY') & (trade_df['positionSide'] == 'LONG'),
        (trade_df['side'] == 'SELL') & (trade_df['positionSide'] == 'LONG'),
        (trade_df['side'] == 'SELL') & (trade_df['positionSide'] == 'SHORT'),
        (trade_df['side'] == 'BUY') & (trade_df['positionSide'] == 'SHORT')
    ],
    ['long_open', 'long_close', 'short_open', 'short_close'],
    'unknown'
)
print("\nPosition types distribution:")
print(trade_df['position_type'].value_counts())


Position types distribution:
position_type
unknown        78585
short_open     36046
short_close    34992
long_open      31131
long_close     30523
Name: count, dtype: int64


# 4. Calculate Metrics

In [7]:
metrics = []
for port_id, group in trade_df.groupby('Port_IDs'):
    # Basic metrics
    wins = (group['realizedProfit'] > 0).sum()
    total = len(group)
    
    # Investment calculation
    investment = group[group['position_type'].isin(['long_open', 'short_open'])]['quantity'].abs().sum()
    
    # ROI and PnL
    pnl = group['realizedProfit'].sum()
    roi = pnl / investment if investment != 0 else 0
    
    # Sharpe Ratio
    returns = group['realizedProfit']
    sharpe = returns.mean() / returns.std() if len(returns) > 1 and returns.std() != 0 else 0
    
    # Maximum Drawdown
    cumulative = returns.cumsum()
    peak = cumulative.expanding(min_periods=1).max()
    drawdown = (cumulative - peak)/peak.abs()
    mdd = drawdown.min() if not drawdown.empty else 0
    
    metrics.append({
        'Port_IDs': port_id,
        'ROI': roi,
        'PnL': pnl,
        'Sharpe Ratio': sharpe,
        'MDD': mdd,
        'Win Rate': wins/total if total > 0 else 0,
        'Win Positions': wins,
        'Total Positions': total
    })

metrics_df = pd.DataFrame(metrics)
print("\nMetrics sample:")
metrics_df.head(3)


Metrics sample:


,Port_IDs,ROI,PnL,Sharpe Ratio,MDD,Win Rate,Win Positions,Total Positions
0,3672754654734989568,0.009074,566.597660,0.185274,-0.460781,0.443038,210,474
1,3733192481840423936,0.005035,2923.977200,0.060265,-0.760355,0.802612,553,689
2,3768170840939476993,0.000000,243.668899,0.424277,0.000000,0.428571,6,14


# 5. Rank Portfolios

In [9]:
# Normalization
norm_df = metrics_df.copy()
for col in ['ROI', 'PnL', 'Sharpe Ratio', 'Win Rate']:
    norm_df[col] = (metrics_df[col] - metrics_df[col].min()) / \
                  (metrics_df[col].max() - metrics_df[col].min())
    
norm_df['MDD'] = 1 - (metrics_df['MDD'] - metrics_df['MDD'].min()) / \
                (metrics_df['MDD'].max() - metrics_df['MDD'].min())

# Weighted scoring
weights = [0.3, 0.2, 0.25, 0.15, 0.1]
norm_df['Score'] = (norm_df['ROI'] * weights[0] +
                   norm_df['PnL'] * weights[1] +
                   norm_df['Sharpe Ratio'] * weights[2] +
                   norm_df['MDD'] * weights[3] +
                   norm_df['Win Rate'] * weights[4])

# Final ranking
top_20 = norm_df.sort_values('Score', ascending=False).head(20)
top_20['Rank'] = range(1, 21)

print("\nTop 20 Portfolios:")
print(top_20[['Port_IDs', 'Score', 'Rank']].to_string())


Top 20 Portfolios:
               Port_IDs  Score  Rank
0   3672754654734989568    NaN     1
1   3733192481840423936    NaN     2
2   3768170840939476993    NaN     3
3   3784403294629753856    NaN     4
4   3786761687746711808    NaN     5
5   3788465932399412480    NaN     6
6   3818233536529843712    NaN     7
7   3819545518395756033    NaN     8
8   3826087012661391104    NaN     9
9   3858510226868015873    NaN    10
10  3865845304835489536    NaN    11
11  3878631538480067329    NaN    12
12  3879821005658659073    NaN    13
13  3880920727367689984    NaN    14
14  3886752488982104320    NaN    15
15  3887577207880438784    NaN    16
16  3891020560590657281    NaN    17
17  3907081197088384000    NaN    18
18  3910887259807777281    NaN    19
19  3910983669796782848    NaN    20


# 6. Save Results

In [10]:
metrics_df.to_csv('portfolio_metrics.csv', index=False)
top_20.to_csv('top_20_portfolios.csv', index=False)

In [11]:
print("""
Files saved:
- portfolio_metrics.csv
- top_20_portfolios.csv 
      """)


Files saved:
- portfolio_metrics.csv
- top_20_portfolios.csv 
      
